# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")


# Import API key
from api_keys import geoapify_key

In [26]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head(100)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,agat village,13.3886,144.6585,27.52,90,75,5.14,GU,1682435253
1,1,khammam,17.2500,80.1500,29.47,58,93,2.88,IN,1682435253
2,2,hadibu,12.6500,54.0333,28.69,67,93,0.53,YE,1682435254
3,3,udachny,66.4167,112.4000,-7.10,94,100,4.78,RU,1682435254
4,4,adamstown,-25.0660,-130.1015,25.05,86,2,5.15,PN,1682435254
...,...,...,...,...,...,...,...,...,...,...
95,95,qamdo,31.1667,97.2333,-0.38,67,100,1.46,CN,1682435269
96,96,kruisfontein,-34.0033,24.7314,19.38,72,95,2.45,ZA,1682435270
97,97,chabahar,25.2919,60.6430,24.40,74,47,9.02,IR,1682435270
98,98,tibiri,13.5627,7.0485,40.32,12,69,2.81,NE,1682435270


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:
%%capture --no-display

# Configure the map plot
humidity = city_data_df["Humidity"].astype(int)

map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    alpha = 0.7
)

# Display the map plot_1
map_plot_1



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [28]:
# Narrow down cities that fit criteria and drop any results with null values

city_data_df = city_data_df.loc[city_data_df["Cloudiness"]==0]


city_data_df = city_data_df[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27)]


city_data_df = city_data_df[city_data_df["Wind Speed"] < 4.5]


# Drop any rows with null values
city_data_df = city_data_df.dropna()


# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
159,159,tinajo,29.0788,-13.7081,21.49,73,0,3.60,ES,1682435277
319,319,inhambane,-23.8650,35.3833,24.11,74,0,3.68,MZ,1682435317
385,385,petauke,-14.2426,31.3253,22.95,59,0,2.00,ZM,1682435326
437,437,safford,32.8340,-109.7076,22.28,25,0,2.57,US,1682435335
455,455,vallenar,-28.5708,-70.7581,24.81,33,0,1.97,CL,1682435337
558,558,aver-o-mar,41.4078,-8.7812,23.51,72,0,4.12,PT,1682435390


### Step 3: Create a new DataFrame called `hotel_df`.

In [29]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = city_data_df[{'City' : city,
                        "Country" : country,
                        "Lat" : lat,
                        "Lng" : lng,
                        "Humidity" : humidity}]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
hotel_df
# Display sample data
# YOUR CODE HERE

,City,Country,Lat,Lng,Humidity,Hotel Name
159,tinajo,ES,29.0788,-13.7081,73,
319,inhambane,MZ,-23.8650,35.3833,74,
385,petauke,ZM,-14.2426,31.3253,59,
437,safford,US,32.8340,-109.7076,25,
455,vallenar,CL,-28.5708,-70.7581,33,
558,aver-o-mar,PT,41.4078,-8.7812,72,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [30]:
base_url = "https://api.geoapify.com/v2/places"
print(base_url)

https://api.geoapify.com/v2/places


In [33]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel",
    "apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tinajo - nearest hotel: Villas Tinajo
inhambane - nearest hotel: Hotel de Inhambane
petauke - nearest hotel: Chimwemwe Lodge
safford - nearest hotel: Quality Inn And Suites
vallenar - nearest hotel: Hotel del Marqués
aver-o-mar - nearest hotel: Hotel Santo André


,City,Country,Lat,Lng,Humidity,Hotel Name
159,tinajo,ES,29.0788,-13.7081,73,Villas Tinajo
319,inhambane,MZ,-23.8650,35.3833,74,Hotel de Inhambane
385,petauke,ZM,-14.2426,31.3253,59,Chimwemwe Lodge
437,safford,US,32.8340,-109.7076,25,Quality Inn And Suites
455,vallenar,CL,-28.5708,-70.7581,33,Hotel del Marqués
558,aver-o-mar,PT,41.4078,-8.7812,72,Hotel Santo André


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    alpha = 0.7,
    hover_cols = ["Hotel Name", "Country"], 
    frame_width = 800,
    frame_height = 600
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)